# Device Integration with Amazon Services

```SQL
CREATE TABLE dim_device (
    device_id INT,
    device_name VARCHAR
);

CREATE TABLE dim_service (
    service_id INT,
    service_name VARCHAR
);

CREATE TABLE fct_device_usage (
    usage_id INT,
    device_id INT,
    service_id INT,
    usage_duration_minutes INT,
    usage_date DATE
);

INSERT INTO dim_device (device_id, device_name)
VALUES
    (1, 'Echo Dot 4th Gen'),
    (2, 'Fire TV Stick 4K'),
    (3, 'Kindle Paperwhite 11th'),
    (4, 'Echo Show 10'),
    (5, 'Fire HD 10 Tablet');

INSERT INTO dim_service (service_id, service_name)
VALUES
    (1, 'Prime Video'),
    (2, 'Amazon Music'),
    (3, 'Alexa Skills');

INSERT INTO fct_device_usage (usage_id, device_id, service_id, usage_date, usage_duration_minutes)
VALUES
    (1, 1, 1, '2024-07-05', 30),
    (2, 2, 1, '2024-07-10', 45),
    (3, 3, 2, '2024-07-15', 60),
    (4, 4, 1, '2024-08-01', 120),
    (5, 5, 2, '2024-08-05', 15),
    (6, 1, 3, '2024-08-10', 25),
    (7, 2, 2, '2024-08-15', 35),
    (8, 3, 1, '2024-09-01', 50),
    (9, 4, 2, '2024-09-05', 80),
    (10, 5, 3, '2024-09-10', 10),
    (11, 1, 1, '2024-07-20', 40),
    (12, 2, 3, '2024-07-25', 20),
    (13, 3, 2, '2024-08-20', 70),
    (14, 4, 1, '2024-08-25', 90),
    (15, 5, 2, '2024-09-15', 55),
    (16, 1, 2, '2024-09-20', 35),
    (17, 2, 1, '2024-09-25', 65),
    (18, 3, 3, '2024-07-30', 5),
    (19, 4, 2, '2024-08-30', 75),
    (20, 5, 1, '2024-09-30', 85);


In [3]:
import pandas as pd
import numpy as np

In [9]:
df_dim_device = pd.read_csv('Data/003/dim_device.csv')
df_dim_service = pd.read_csv('Data/003/dim_service.csv')
df_fct_usage = pd.read_csv('Data/003/fct_device_usage.csv')

In [5]:
df_dim_device.head()

,device_id,device_name
0,1,Echo Dot 4th Gen
1,2,Fire TV Stick 4K
2,3,Kindle Paperwhite 11th
3,4,Echo Show 10
4,5,Fire HD 10 Tablet


In [6]:
df_dim_service.head()

,service_id,service_name
0,1,Prime Video
1,2,Amazon Music
2,3,Alexa Skills


In [10]:
df_fct_usage.head()

,usage_id,device_id,service_id,usage_duration_minutes,usage_date
0,1,1,1,30,2024-07-05
1,2,2,1,45,2024-07-10
2,3,3,2,60,2024-07-15
3,4,4,1,120,2024-08-01
4,5,5,2,15,2024-08-05


# Pregunta 1

### El equipo desea identificar la duración total de uso de los servicios para cada tipo de dispositivo, extrayendo la categoría principal del dispositivo a partir de su nombre, para el periodo del 1 de julio de 2024 al 30 de septiembre de 2024. La categoría principal del dispositivo se obtiene de la primera palabra del nombre del dispositivo.

In [11]:
# 1. Preparar las fechas en la tabla de hechos
df_fct_usage['usage_date'] = pd.to_datetime(df_fct_usage['usage_date'])

# 2. Extraer la categoría principal (primera palabra) del nombre del dispositivo
# .str.split(' ').str[0] equivale al SPLIT_PART de SQL
df_dim_device['device_category'] = df_dim_device['device_name'].str.split(' ').str[0]

# 3. Filtrar los datos para el periodo solicitado (Q3 2024)
q3_usage = df_fct_usage[
    (df_fct_usage['usage_date'] >= '2024-07-01') & 
    (df_fct_usage['usage_date'] <= '2024-09-30')
]

# 4. Unir (Merge) las tablas
# Unimos hechos con dispositivos para tener la categoría, y luego con servicios para el nombre del servicio
merged_df = pd.merge(q3_usage, df_dim_device, on='device_id')
merged_df = pd.merge(merged_df, df_dim_service, on='service_id')

# 5. Agrupar y sumar la duración
result = (
    merged_df.groupby(['device_category', 'service_name'])['usage_duration_minutes']
    .sum()
    .reset_index()
    .sort_values(by=['device_category', 'usage_duration_minutes'], ascending=[True, False])
)

print(result)

  device_category  service_name  usage_duration_minutes
2            Echo   Prime Video                     280
1            Echo  Amazon Music                     190
0            Echo  Alexa Skills                      25
5            Fire   Prime Video                     195
4            Fire  Amazon Music                     105
3            Fire  Alexa Skills                      30
7          Kindle  Amazon Music                     130
8          Kindle   Prime Video                      50
6          Kindle  Alexa Skills                       5


```SQL
SELECT
    SPLIT_PART(d.device_name, ' ', 1) AS device_category,
    s.service_name,
    SUM(f.usage_duration_minutes) AS total_usage_duration
FROM fct_device_usage f
JOIN dim_device d ON f.device_id = d.device_id
JOIN dim_service s ON f.service_id = s.service_id
WHERE f.usage_date BETWEEN '2024-07-01' AND '2024-09-30'
GROUP BY device_category, s.service_name
ORDER BY device_category, total_usage_duration DESC;
```

# Pregunta 2

### El equipo también desea etiquetar el uso de cada categoría de dispositivo como 'Bajo' (Low) o 'Alto' (High) basándose en la duración del uso desde el 1 de julio de 2024 hasta el 30 de septiembre de 2024. Si el tiempo total de uso fue menor a 300 minutos, lo categorizaremos como 'Low'. De lo contrario, lo categorizaremos como 'High'. ¿Puedes devolver un informe con el ID del dispositivo, la categoría de uso y el tiempo total de uso?

In [12]:
import pandas as pd
import numpy as np

# 1. Asegurar formato de fecha
df_fct_usage['usage_date'] = pd.to_datetime(df_fct_usage['usage_date'])

# 2. Filtrar por el rango de fechas (Q3 2024)
q3_data = df_fct_usage[
    (df_fct_usage['usage_date'] >= '2024-07-01') & 
    (df_fct_usage['usage_date'] <= '2024-09-30')
]

# 3. Agrupar por device_id y sumar los minutos
report = q3_data.groupby('device_id')['usage_duration_minutes'].sum().reset_index()

# 4. Renombrar columna para claridad
report = report.rename(columns={'usage_duration_minutes': 'total_usage_time'})

# 5. Aplicar la lógica condicional (Equivalente al CASE WHEN)
# Si el tiempo es < 300 es 'Low', de lo contrario es 'High'
report['usage_category'] = np.where(report['total_usage_time'] < 300, 'Low', 'High')

# 6. Ordenar resultados
result = report.sort_values(by='total_usage_time', ascending=False)

print(result)

   device_id  total_usage_time usage_category
3          4               365           High
2          3               185            Low
1          2               165            Low
4          5               165            Low
0          1               130            Low


``` SQL
SELECT 
    device_id,
    SUM(usage_duration_minutes) AS total_usage_time,
    CASE 
        WHEN SUM(usage_duration_minutes) < 300 THEN 'Low'
        ELSE 'High'
    END AS usage_category
FROM fct_device_usage
WHERE usage_date BETWEEN '2024-07-01' AND '2024-09-30'
GROUP BY device_id
ORDER BY total_usage_time DESC;
```

# Pregunta 3

### El equipo está considerando empaquetar las suscripciones de Prime Video y Amazon Music. Quieren entender qué porcentaje del tiempo total de uso proviene de los servicios Prime Video y Amazon Music respectivamente. Por favor, utiliza los datos del periodo del 1 de julio de 2024 al 30 de septiembre de 2024

In [13]:
import pandas as pd

# 1. Preparar fechas
df_fct_usage['usage_date'] = pd.to_datetime(df_fct_usage['usage_date'])

# 2. Filtrar por el rango de fechas solicitado (Q3 2024)
q3_mask = (df_fct_usage['usage_date'] >= '2024-07-01') & (df_fct_usage['usage_date'] <= '2024-09-30')
df_q3 = df_fct_usage[q3_mask].copy()

# 3. Calcular el Gran Total de minutos en el Q3 (todos los servicios)
grand_total_minutes = df_q3['usage_duration_minutes'].sum()

# 4. Unir con la dimensión de servicios para filtrar por nombre
df_q3_named = pd.merge(df_q3, df_dim_service, on='service_id')

# 5. Filtrar solo los servicios de interés: Prime Video y Amazon Music
bundle_services = df_q3_named[df_q3_named['service_name'].isin(['Prime Video', 'Amazon Music'])]

# 6. Agrupar por servicio, sumar minutos y calcular porcentaje
result = bundle_services.groupby('service_name')['usage_duration_minutes'].sum().reset_index()

# Calculamos el porcentaje respecto al total global del Q3
result['percentage_of_total'] = (result['usage_duration_minutes'] / grand_total_minutes) * 100

# Redondear para una mejor lectura
result['percentage_of_total'] = result['percentage_of_total'].round(2)

print(result)

   service_name  usage_duration_minutes  percentage_of_total
0  Amazon Music                     425                42.08
1   Prime Video                     525                51.98


```SQL
WITH total_q3 AS (
    -- Calculamos el gran total de minutos de todos los servicios en Q3
    SELECT SUM(usage_duration_minutes) as grand_total
    FROM fct_device_usage
    WHERE usage_date BETWEEN '2024-07-01' AND '2024-09-30'
)
SELECT 
    s.service_name,
    SUM(f.usage_duration_minutes) AS service_usage_time,
    -- Calculamos el porcentaje: (suma_servicio / gran_total) * 100
    ROUND((SUM(f.usage_duration_minutes) * 100.0 / (SELECT grand_total FROM total_q3)), 2) AS percentage_of_total
FROM fct_device_usage f
JOIN dim_service s ON f.service_id = s.service_id
WHERE f.usage_date BETWEEN '2024-07-01' AND '2024-09-30'
  AND s.service_name IN ('Prime Video', 'Amazon Music')
GROUP BY s.service_name;
```